In [98]:
import numpy as np
from qiskit import *
%matplotlib inline
from qiskit.visualization import plot_histogram
from qiskit.tools.visualization import circuit_drawer
from qiskit.circuit import Gate

In [7]:
def apply_gate(circuit,gate_str,applied,ctrl=0):
    if gate_str == 'H':
        circuit.h(applied)
    elif gate_str == 'HZ':
        circuit.z(applied)
        circuit.h(applied)
    elif gate_str == 'X':
        circuit.x(applied)
    elif gate_str == 'Z':
        circuit.z(applied)
    elif gate_str == 'CX':
        circuit.cx(ctrl,applied)
    elif gate_str == 'I':
        pass
    return circuit

In [88]:
gates_lists = ['H', 'HZ', 'X', 'Z', 'CX'] + 3*['I']

def generate_subcircuit(no_qubits):
    circuit = QuantumCircuit(no_qubits)
    for q in range(no_qubits):
        gate = np.random.choice(gates_lists)
        ctrl = 0
        if gate == 'CX':
            list_qubits = list(range(no_qubits))
            list_qubits.pop(q)
            ctrl = np.random.choice(list_qubits)
        apply_gate(circuit, gate, q, ctrl)
    circuit.barrier()
    return circuit

In [90]:
circuit = generate_subcircuit(2)
circuit.draw()

In [94]:
def generate_game(no_qubits, no_rounds):
    Circuits = []
    Plays = []
    
    init_circuit = QuantumCircuit(no_qubits)
    init_circuit.h(range(no_qubits))
    init_circuit.barrier()
    Circuits.append(init_circuit)
    
    for n in range(no_rounds):
        qubits_play = np.random.choice(list(range(no_qubits)),2,replace = False)
        np.random.shuffle(qubits_play)
        play = [('P0',qubits_play[0]), ('P1',qubits_play[1])]      
        Plays.append(play)
        Circuits.append(generate_subcircuit(no_qubits))
        
    return Circuits, Plays

In [238]:
def distribute_cards(no_rounds):
    cards_list = ['H', 'HZ', 'X', 'Z']
    deck = np.array(no_rounds*cards_list)
    np.random.shuffle(deck)
    hand_size = no_rounds + 1
    hand_P0 = deck[0:hand_size]
    hand_P1 = deck[hand_size:2*hand_size]
    return hand_P0, hand_P1

In [226]:
def draw_game(Circuits, Plays):
    final_circ = Circuits[0].copy()
    for play, circ in zip(Plays, Circuits[1:]):
        for player, qubit in play:
            if player in ['P0','P1']:
                final_circ.append(Gate(name = player, num_qubits = 1, params = []),[qubit])
            else:
                final_circ = apply_gate(final_circ,gate_str = player,applied = qubit)
        final_circ.barrier()
        final_circ = final_circ + circ
    print(final_circ.draw())

In [222]:
Circuits, Plays = generate_game(2,2)
Plays[0] = [('X',1),('H',0)]
draw_game(Circuits, Plays)

        ┌───┐ ░ ┌───┐ ░  ░ ┌────┐ ░            ░ 
q_0: |0>┤ H ├─░─┤ H ├─░──░─┤ P1 ├─░────────────░─
        ├───┤ ░ ├───┤ ░  ░ ├────┤ ░ ┌───┐┌───┐ ░ 
q_1: |0>┤ H ├─░─┤ X ├─░──░─┤ P0 ├─░─┤ Z ├┤ H ├─░─
        └───┘ ░ └───┘ ░  ░ └────┘ ░ └───┘└───┘ ░ 


In [137]:
Plays

[[('Z', 1), ('X', 0)], [('P0', 1), ('P1', 0)]]

In [132]:
def play_round(n_round,Plays,P0,P1):
    Plays[n_round][0] = (P0, Plays[n_round][0][1])
    Plays[n_round][1] = (P1, Plays[n_round][1][1])
    return Plays

In [220]:
def get_played_game(Circuits, Plays):
    final_circ = Circuits[0].copy()
    print(final_circ.draw())
    for play, circ in zip(Plays, Circuits[1:]):
        for player, qubit in play:
            if player in ['P0','P1']:
                return final_circ
            else:
                final_circ = apply_gate(final_circ, gate_str = player,applied = qubit)         
        final_circ.barrier()
        final_circ = final_circ + circ
    return final_circ

# def get_played_game(Circuits, Plays):
#     final_circ = Circuits[0].copy()
#     print(final_circ.draw())
#     for play, circ in zip(Plays, Circuits[1:]):
#         for player, qubit in play:
#             if player in ['P0','P1']:
#                 return final_circ
#             else:
#                 final_circ = apply_gate(final_circ, gate_str = player,applied = qubit)         
#         final_circ.barrier()
#         final_circ = final_circ + circ
#     return final_circ

In [225]:
partial_circ = get_played_game(Circuits, Plays)
partial_circ.draw()

        ┌───┐ ░ ┌───┐ ░ 
q_0: |0>┤ H ├─░─┤ H ├─░─
        ├───┤ ░ ├───┤ ░ 
q_1: |0>┤ H ├─░─┤ X ├─░─
        └───┘ ░ └───┘ ░ 


In [210]:
def compute_state(partial_circuit):
    backend = Aer.get_backend('statevector_simulator')
    job = execute(partial_circ, backend)
    result = job.result()
    state = result.get_statevector(partial_circ, decimals=3)
    return state

In [204]:
def score_counts(state_v):
    e_ones = 0
    for i, p in enumerate(np.abs(state_v)**2):
        e_ones += p*np.sum(np.array(list(bin(i)[2:])).astype(np.int))
    return np.around(e_ones/np.log2(len(state_v))*100,decimals = 1)

In [179]:
def trace_distance(A,B):
    X = A-B
    evals = np.linalg.eigvals(X)
    return 0.5*np.sum(np.abs(evals))

In [184]:
def score_tr_dist(state_v):
    rho = np.tensordot(state_v, state_v.conj(), axes = 0)
    rho_1 = np.zeros_like(rho)
    rho_1[-1,-1] = 1
    return trace_distance(rho, rho_1)

In [191]:
#score_tr_dist(np.array([0.25,0.25,0.25,0.25]))
score_tr_dist(np.array([0,np.sqrt(0.5),np.sqrt(0.5),0]))

1.0